# LLMRouterChain

In [5]:
import openai
import os 
import getpass
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [7]:
os.environ['OPENAI_API_KEY'] = getpass.getpass()

 ········


In [8]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [9]:
llm = ChatOpenAI()

C:\Users\bharanidharan.a\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
hr_template="""You are a very smart human resources person. \
You are great at answering questions about human resources in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


In [11]:
law_template="""You are a very smart lawyer. \
You are great at answering questions about the law in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

In [12]:
prompt_infos = [
{
    'name':'human resources',
    'description':'Answer human resources questions',
    'template':hr_template},

{
    'name':'law',
    'description':'Answers legal questions',
    'template':law_template}
]

In [13]:
destination_chains ={}

In [14]:
for p_info in prompt_infos:
    name = p_info['name']
    prompt_template = p_info['template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm,prompt=prompt)
    destination_chains[name] = chain

C:\Users\bharanidharan.a\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [15]:
default_prompt = ChatPromptTemplate.from_template('{input}')

default_chain = LLMChain(llm=llm,prompt=default_prompt)

In [16]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [17]:

destinations=[
f"{p['name']}: {p['description']}" for p in prompt_infos
]


In [19]:
destinations

['human resources: Answer human resources questions',
 'law: Answers legal questions']

In [20]:
destination_str = "\n".join(destinations)

print(destination_str)

human resources: Answer human resources questions
law: Answers legal questions


In [21]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)

router_template

'Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nhuman resources: Answer human resources questions\nlaw: Answers legal questions\n\

In [23]:
router_prompt = PromptTemplate(template=router_template,
input_variables=['input'],
output_parser=RouterOutputParser())

router_chain = LLMRouterChain.from_llm(llm,router_prompt)

chain = MultiPromptChain(router_chain=router_chain,
destination_chains=destination_chains,
default_chain=default_chain,verbose=True)

chain.run("How do intellectual property laws address the use of AI-generated content?")



> Entering new MultiPromptChain chain...
law: {'input': 'How do intellectual property laws address the use of AI-generated content?'}
> Finished chain.


'Thank you for the compliment. \n\nIntellectual property laws are still evolving to address the use of AI-generated content. In general, copyright law protects original works of authorship, but it can be challenging to determine who the author is when content is generated by AI. Some argue that the creator of the AI should be considered the author, while others argue that the user who directs the AI should be considered the author. \n\nAs of now, there is no clear consensus on how to address AI-generated content under intellectual property laws. It is a complex and rapidly evolving area of law that will likely require further guidance and legislation in the future.'

In [24]:
chain.run("How do you handle conflict between team members?")



> Entering new MultiPromptChain chain...
human resources: {'input': 'How to effectively handle conflict between team members?'}
> Finished chain.


'Handling conflict between team members effectively involves several key steps:\n\n1. Address the issue promptly: As soon as you become aware of a conflict, address it promptly to prevent it from escalating further.\n\n2. Encourage open communication: Encourage team members to openly communicate their concerns and perspectives on the conflict. Encourage active listening and empathetic understanding.\n\n3. Identify the root cause: Work with the involved parties to identify the underlying issues causing the conflict. Understanding the root cause can help in resolving the conflict effectively.\n\n4. Facilitate a resolution: Work with the team members involved to find a resolution that is agreeable to all parties. Encourage compromise and collaboration to find a solution that works for everyone.\n\n5. Follow up: After the conflict has been resolved, follow up with the team members involved to ensure that the resolution is working and that any lingering issues are addressed.\n\nRemember, ef